In [2]:
import pandas as pd
import numpy as np
import unidecode
import re
from shapely.geometry import Point, Polygon
import folium
from folium.plugins import HeatMap

In [3]:
import geopandas as gpd

In [4]:
fname = "./data/loc.geojson"

df = gpd.read_file(fname)

print(type(df))

df.head()

<class 'geopandas.geodataframe.GeoDataFrame'>


,LOC,geometry
0,ANTONIO NARIÑO,"POLYGON ((-74.13075446257029 4.59335334662506,..."
1,LA CANDELARIA,"POLYGON ((-74.0662087260469 4.60317174214195, ..."
2,CHAPINERO,"POLYGON ((-74.0572470317838 4.68683629566918, ..."
3,CIUDAD BOLIVAR,"POLYGON ((-74.15216009957931 4.59975861479746,..."
4,ENGATIVA,"POLYGON ((-74.127379945115 4.73730502762603, -..."


In [5]:
re.sub(r'I', '', "PABLO I")

'PABLO '

In [6]:
df_hurtos = pd.read_csv("./data/homicidios_bogota_2018.csv")
len(df_hurtos)

431

In [7]:
df_hurtos.head()

,Fecha,Departamento,Municipio,Día,Hora,Barrio,Zona,Clase de sitio,Arma empleada,Móvil Agresor,Móvil Victima,Edad,Sexo,Estado civil,País de nacimiento,Clase de empleado,Profesión,Escolaridad,Código DANE,Cantidad
0,01/01/2018 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Lunes,1:00:00,SANTA LIBRADA E-5,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,A PIE,22.0,FEMENINO,SOLTERO,COLOMBIA,DESEMPLEADO,-,SECUNDARIA,11001000.0,1
1,01/01/2018 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Lunes,5:30:00,LOS TRES REYES - I ETAPA E-19,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,A PIE,17.0,MASCULINO,SOLTERO,COLOMBIA,DESEMPLEADO,-,SECUNDARIA,11001000.0,1
2,01/01/2018 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Lunes,2:30:00,SAN ANTONIO E-15,URBANA,CASAS DE HABITACION,ARMA BLANCA / CORTOPUNZANTE,A PIE,A PIE,30.0,MASCULINO,UNION LIBRE,COLOMBIA,INDEPENDIENTE,-,SECUNDARIA,11001000.0,1
3,01/01/2018 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Lunes,2:35:00,CENTRO USME E-5,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,A PIE,24.0,MASCULINO,UNION LIBRE,COLOMBIA,EMPLEADO PARTICULAR,-,SECUNDARIA,11001000.0,1
4,01/01/2018 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Lunes,7:45:00,LA PRIMAVERA E-7,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,A PIE,43.0,MASCULINO,UNION LIBRE,COLOMBIA,DESEMPLEADO,-,SECUNDARIA,11001000.0,1


In [8]:
df_barrios = pd.read_csv("./data/barrios_bogota.csv")

In [9]:
df_barrios.head()

,OBJECTID,Identificador unico del sector catastral,Tipo de sector catastral,Nombre del sector catastral,coord_x,coord_y
0,1281,205109,2,LA MAGDALENA I,"-74,15971638349998","4,6538848121528105"
1,1282,205227,2,OSORIO IX,"-74,171430826","4,655330582652857"
2,1284,105203,1,OSORIO II,"-74,166000932","4,654956388652853"
3,1285,201109,2,INGEMAR I,"-74,05440703649998","4,639504332652364"
4,1286,201110,2,INGEMAR ORIENTAL I,"-74,0505909105","4,644192807152489"


## Limpiar columna "barrio"

In [10]:
UMBRAL = 0

In [11]:
def arreglar_I(x):
    if " I" in x[-2:]:
        return x[:-2]
    if " II" in x[-3:]:
        return x[:-3]
    if " III" in x[-4:]:
        return x[:-4]
    else:
        return x

In [12]:
def arreglar_loc_hurtos(x):
    try:
        return x.split(" E-")[1]
    except:
        print(x)

In [13]:
df_hurtos["Localidad"] = df_hurtos["Barrio"].apply(arreglar_loc_hurtos)

In [14]:
df_hurtos["Barrio"] = df_hurtos["Barrio"].apply(lambda x:x.split(" E-")[0])
df_hurtos["Barrio"] = df_hurtos["Barrio"].apply(lambda x:x.replace(" II "," ").replace(" I "," "))
df_hurtos["Barrio"] = df_hurtos["Barrio"].apply(arreglar_I)
df_hurtos["Barrio"] = df_hurtos["Barrio"].apply(lambda x:x
                                                                                            .replace("LOS ","")
                                                                                            .replace("LAS ","")
                                                                                            .replace("EL ","")
                                                                                            .replace("LA ","")
                                                                                           )
df_hurtos["Barrio"] = df_hurtos["Barrio"].apply(lambda x:x
                                                                                            .replace(" NOR-ORIENTAL","")
                                                                                            .replace(" SUR-ORIENTAL","")
                                                                                            .replace(" NOR-OCCIDENTAL","")
                                                                                            .replace(" SUR-OCCIDENTAL","")
                                                                                            .replace(" OCCIDENTAL","")
                                                                                            .replace(" ORIENTAL","")
                                                                                            .replace(" SUR","")
                                                                                            .replace(" NORTE","")
                                                                                            .replace(" CENTRO","")
                                                                                            .replace(" BAJO","")
                                                                                            .replace(" ALTO","")
                                                                                            .replace(" CENTRAL","")
                                                                                            .replace(" ETAPA","")
                                                                                            .replace(" SECTOR","")
                                                                                           )

In [15]:
df_barrios["Nombre del sector catastral"] = df_barrios["Nombre del sector catastral"].apply(lambda x:unidecode.unidecode(x))
df_barrios["Nombre del sector catastral"] = df_barrios["Nombre del sector catastral"].apply(lambda x:x.replace(" II","").replace(" I",""))
"""df_barrios["Nombre del sector catastral"] = df_barrios["Nombre del sector catastral"].apply(lambda x:x
                                                                                            .replace(" NOR-ORIENTAL","")
                                                                                            .replace(" SUR-ORIENTAL","")
                                                                                            .replace(" NOR-OCCIDENTAL","")
                                                                                            .replace(" SUR-OCCIDENTAL","")
                                                                                            .replace(" OCCIDENTAL","")
                                                                                            .replace(" ORIENTAL","")
                                                                                            .replace(" SUR","")
                                                                                            .replace(" NORTE","")
                                                                                           )"""
df_barrios["Nombre del sector catastral"] = df_barrios["Nombre del sector catastral"].apply(lambda x:x
                                                                                            .replace("LOS ","")
                                                                                            .replace("LAS ","")
                                                                                            .replace("EL ","")
                                                                                            .replace("LA ","")
                                                                                           )

In [16]:
np.size(df_hurtos["Barrio"].value_counts().loc[df_hurtos["Barrio"].value_counts()>0])

279

In [17]:
barrios_con_umbral = np.size(df_hurtos["Barrio"].value_counts().loc[df_hurtos["Barrio"].value_counts()>UMBRAL])

In [18]:
barrios_hurtos = pd.Series((df_hurtos["Barrio"].value_counts().loc[df_hurtos["Barrio"].value_counts()>UMBRAL]).index)
barrios_hurtos.head()

0             FAVORITA
1             CARACOLI
2           MARGARITAS
3        GRAN BRITALIA
4    ARBORIZADORA ALTA
dtype: object

In [19]:
def arreglar(x):
    resp = ""
    for barrio in barrios_hurtos:
        if barrio in x:
            return barrio
    return x

In [20]:
df_barrios["Nombre del sector catastral"] = df_barrios["Nombre del sector catastral"].apply(arreglar)

In [21]:
barrios_catastro = pd.Series(df_barrios["Nombre del sector catastral"].unique())

In [22]:
barrios_coincidencia = np.sum(barrios_hurtos.isin(barrios_catastro))

In [23]:
print("Barrios: %s" % barrios_con_umbral)
print("Barrios coincidentes: %s" % barrios_coincidencia)
print("% Coincidencia:")
print(barrios_coincidencia/barrios_con_umbral)

Barrios: 279
Barrios coincidentes: 178
% Coincidencia:
0.6379928315412187


In [24]:
df_hurtos.loc[df_hurtos["Barrio"].isin(barrios_hurtos.loc[~barrios_hurtos.isin(barrios_catastro)])]["Barrio"].value_counts()

CANDELARIA III-IV          3
MADELENA Y                 3
GUACAMAYAS I-II            2
CABAÑA                     2
CARVAJAL                   2
INDUSTRIAL PENSILVANIA     2
VISTA HERMOSA CAPRI        2
VEREDA TIBAQUE             2
BILBAO                     2
JAPON                      2
VILDE COMUNEROS            2
SAN JOSE A                 2
SAN BERNARDO               2
MANZANARES                 2
MILLÁN SAUCES              2
PEÑON DCORTIJO             2
ALQUERIAS DE FRAGUA        2
GAITANA                    2
POA                        2
VILHERMOSA                 2
GRANJAS DE SAN PABLO       1
COMPOSTELA                 1
CENTRO INDUSTRIAL          1
VILDDORADO                 1
LIBERTADOR                 1
ESTRELLITA                 1
DIVINO ROSTRO              1
CANTALEJO                  1
VISTA HERMOSA ESPINAL      1
RINCON DE BRITALIA         1
                          ..
SAN ANTONIO DE ENGATIVA    1
BOSQUE IZQUIERDO           1
TIBABITA                   1
TUNALITO      

In [25]:
np.sum(~barrios_hurtos.isin(barrios_catastro))

101

In [26]:
df_barrios["coord_x"] = df_barrios["coord_x"].apply(lambda x:float(x.replace(",",".")))
df_barrios["coord_y"] = df_barrios["coord_y"].apply(lambda x:float(x.replace(",",".")))

In [27]:
dict_localidades = {
    'KENNEDY':"8",
    'CHAPINERO':"2",
    'SAN CRISTOBAL':"4",
    'SANTA FE':"3",
    'USME':"5",
    'RAFAEL URIBE URIBE':"18",
    'CIUDAD BOLIVAR':"19",
    'ENGATIVA':"10",
    'TEUSAQUILLO':"13",
    'PUENTE ARANDA':"16",
    'LOS MARTIRES':"14",
    'BOSA':"7",
    'ANTONIO NARIÑO':"15",
    'USAQUEN':"1",
    'FONTIBON':"9",
    'BARRIOS UNIDOS':"12",
    'TUNJUELITO':"6",
    'SUBA':"11"
}

In [28]:
def get_localidad(x,y):
    try:
        resp = ""
        p = Point(x,y)
        for i in df.iterrows():
            loc_name = i[1]["LOC"]
            loc_pol = i[1]["geometry"]
            if p.within(loc_pol):
                resp = loc_name
        return dict_localidades[resp]
    except:
        print(x, y)

In [29]:
df_barrios["Localidad"] = df_barrios.apply(lambda x:get_localidad(x["coord_x"], x["coord_y"]), axis=1)

-74.01923528699999 4.661020825153022


In [30]:
df_barrios["Localidad"].unique()

array(['8', '2', '4', '3', '5', '18', '19', '10', '13', '16', '14', '7',
       '15', '1', '9', '12', '6', '11', None], dtype=object)

In [31]:
len(dict_localidades)

18

In [32]:
df_hurtos["llave-barrio"] = df_hurtos["Barrio"] + "-" + df_hurtos["Localidad"]

In [33]:
df_hurtos.head(2)

,Fecha,Departamento,Municipio,Día,Hora,Barrio,Zona,Clase de sitio,Arma empleada,Móvil Agresor,...,Sexo,Estado civil,País de nacimiento,Clase de empleado,Profesión,Escolaridad,Código DANE,Cantidad,Localidad,llave-barrio
0,01/01/2018 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Lunes,1:00:00,SANTA LIBRADA,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,...,FEMENINO,SOLTERO,COLOMBIA,DESEMPLEADO,-,SECUNDARIA,11001000.0,1,5,SANTA LIBRADA-5
1,01/01/2018 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Lunes,5:30:00,TRES REYES -,URBANA,VIAS PUBLICAS,ARMA BLANCA / CORTOPUNZANTE,A PIE,...,MASCULINO,SOLTERO,COLOMBIA,DESEMPLEADO,-,SECUNDARIA,11001000.0,1,19,TRES REYES --19


In [34]:
df_barrios["llave-barrio"] = df_barrios["Nombre del sector catastral"] + "-" + df_barrios["Localidad"]

In [35]:
df_barrios = df_barrios.groupby("llave-barrio").mean()[["coord_x", "coord_y"]].reset_index()

In [36]:
df_count = df_hurtos.groupby("llave-barrio").count()["Fecha"].to_frame().reset_index()

In [37]:
df_def = pd.merge(df_count, df_barrios, on="llave-barrio", how="inner")

In [38]:
len(df_hurtos)

431

In [39]:
df_def["Fecha"].sum()

272

In [40]:
df_def.to_csv("./data/base_definitiva.csv", index=False)

In [41]:
max_amount = float(df_def['Fecha'].max())

In [42]:
hmap = folium.Map(location=[4.6, -74.1], zoom_start=11)

In [43]:
df_def.Fecha = df_def.Fecha.apply(lambda x:int(str(x)))

In [44]:
hm_wide = HeatMap( list(zip(df_def.coord_x.values, df_def.coord_y.values, [int(x) for x in list(df_def.Fecha.values)])),
                   min_opacity=0.2,
                   max_val=max_amount,
                   radius=17, blur=15, 
                   max_zoom=1, 
                 )

In [45]:
hm_wide = HeatMap( list(zip(df_def.coord_y.values, df_def.coord_x.values, [int(x) for x in list(df_def.Fecha.values)])),
                   min_opacity=0.2,
                   max_val=max_amount,
                   radius=17, blur=15, 
                   max_zoom=1, 
                 )

In [46]:
hmap.add_child(hm_wide)

In [47]:
hmap.save('mapa_homicidios.html')